<div style="border:solid light orange 3px; padding: 16px">  
    <font size="4">  
        <p style="text-align: center;">
            <b> 2. Предобработка данных. Исследование надежности замещиков </b>
        </p> 
    </font>
</div>

## 0. Описание проекта 

Заказчик — кредитный отдел банка. Входные данные от банка — статистика о платёжеспособности клиентов. Необходимо разобраться, влияет ли семейное положение, количество детей и прочие характерные черты клиента на факт погашения кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## 1. Получение данных
Изучим данные, полученные от банка.

In [1]:
import pandas as pd

path = 'C:/Users/pavel/Desktop/da/projects/datasets/'

In [2]:
# прочтём файл с данными и сохраним его содержимое в df
df = pd.read_csv(path + '02_credit_data.csv')

In [3]:
# выведем первые 5 строк таблицы df
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Описание данных:

- `children` — количество детей в семье;
- `days_employed` — трудовой стаж в днях;
- `dob_years` — возраст клиента в годах;
- `education` — образование клиента;
- `education_id` — идентификатор образования;
- `family_status` — семейное положение;
- `family_status_id` — идентификатор семейного положения;
- `gender` — пол клиента;
- `income_type` — тип занятости;
- `debt` — имел ли задолженность по возврату кредитов;
- `total_income` — доход в месяц;
- `purpose` — цель получения кредита.

In [4]:
# вывод информации о размере таблицы
df.shape

(21525, 12)

In [5]:
#вывод общей информации о таблице (тип данных в столбце, кол-во записей в столбце)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## 2. Предобработка данных
Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

Из общей информации видно, что пропуски есть только в столбцах `days_employed` и `total_income`.

In [6]:
# Количество пропусков в столбцах:
print("Количество пропусков в столбце 'days_employed' (трудовой стаж):", 
      df['days_employed'].isna().sum())

print("Количество пропусков в столбце 'total_income' (доход в месяц):", 
      df['total_income'].isna().sum())

Количество пропусков в столбце 'days_employed' (трудовой стаж): 2174
Количество пропусков в столбце 'total_income' (доход в месяц): 2174


In [7]:
# выделим строки с пропусками
df[df['days_employed'].isna() == True].head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Проверим, есть ли строки с пропущенным стажем, но указанным доходом.

In [8]:
df[(df['days_employed'].isna() == True) & (df['total_income'].isna() == False)]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Таких строк нет (если не указан трудовой стаж, то не указана и зарплата), можно предположить что эти пропуски связаны, т.е. не случайны. Например, так могут вноситься клиенты банка, которые не подтвердили место работы или доход (устроенные неофициально).

---

В столбцах `education`, `family_status`, `gender`, `income_type` содержатся данные, которые можно разделить на несколько категорий. Посмотрим, какие уникальные значения содержатся в каждом столбце.

In [9]:
# уникальные значения в столбце 'education' и их количество
df['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Выделяются несколько значений, которые отличаются только регистром. 

In [10]:
# приведем все символы к нижнему регистру
df['education'] = df['education'].str.lower()

# снова выведем список уникальных значений
df['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Остались только 5 категорий, которые отличаются друг от друга по смыслу.

Проверим остальные столбцы с категориальными данными.

---

In [11]:
df['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

---

In [12]:
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

Даже если предположить, что XNA это не ошибка при вводе данных, а отдельная категория "неуказанный/невыбранный пол", удалим эту запись, т.к. 1 значение из ~21000 не значимо, но в дальнейшем может сделать анализ неудобным.


In [13]:
# оставим в датасете только те записи, у которых значение в столбце 'gender' не равно 'XNA'
df = df[df['gender'] != 'XNA']

# перенумеруем индексы
df = df.reset_index(drop = 'True')

# проверим список уникальных значений в столбце 'gender' и их количество
print(df['gender'].value_counts())

F    14236
M     7288
Name: gender, dtype: int64


---

In [14]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

Удалим записи со значениями `предприниматель`, `безработный`, `студент`,`в декрете` в столбце `income_type`. Количество записей мало, поэтому удаление не повлияет на итоговые выводы существенным образом.

In [15]:
# оставим в датасете только те записи, у которых значение в столбце 'income_type' не равно 
# 'предприниматель' или 'безработный' или 'студент' или 'в декрете'
df = df[(df['income_type'] != 'предприниматель') & \
      (df['income_type'] != 'безработный') & \
      (df['income_type'] != 'студент') & \
      (df['income_type'] != 'в декрете')]

# перенумеруем индексы
df = df.reset_index(drop = 'True')

# проверим список уникальных значений в столбце 'income_type' и их количество
print(df['income_type'].value_counts())

сотрудник      11119
компаньон       5084
пенсионер       3856
госслужащий     1459
Name: income_type, dtype: int64


---

Для заполнения пропущенных значений в столбцах `days_employed` будем использовать средние значения трудового стажа для людей такого же возраста и такого же образования. 

Для этого создадим функцию, принимающую на вход значение возраста и категорию образования, и возвращающую среднее значение трудового стажа, подсчитанное для клиентов (с указанным трудовым стажем) с заданным возрастом и образованием.

In [16]:
def mean_days_employed(row):
    years = row['dob_years']
    education = row['education']
    return df[(df['days_employed'].isna() == False) \
              & (df['dob_years'] == years) \
              & (df['education'] == education)]['days_employed'].mean()

In [17]:
# создадим новый столбец, как результат применения функции к каждой строке в датасете
df['mean_days_employed'] = df.apply(mean_days_employed, axis = 1)

In [18]:
df.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,mean_days_employed
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,2374.265767
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,-1263.661373
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,339.142890
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1673.410241
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,91415.560024
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,-1322.509097
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7070.099300
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,49954.425498
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,-2034.546455
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,2425.889224


In [19]:
# новый столбец, равный столбцу 'days_employed', но с заполненными пропусками 
df['days_employed_0'] = df['days_employed'].fillna(0)

# записываем значения столбца 'days_employed', как старые, в случае, если пропуска в столбце нет (.isna() возвращает 0)
# и как среднее значение трудового стажа для клиентов такого же возраста с таким же образованием, в случае пропуска
df['days_employed'] = \
df['days_employed_0'] + df['days_employed'].isna() * df['mean_days_employed']

In [20]:
df.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,mean_days_employed,days_employed_0
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,2374.265767,-8437.673028
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,-1263.661373,-4024.803754
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,339.142890,-5623.422610
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1673.410241,-4124.747207
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,91415.560024,340266.072047
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,-1322.509097,-926.185831
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7070.099300,-2879.202052
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,49954.425498,-152.779569
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,-2034.546455,-6929.865299
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,2425.889224,-2188.756445


---

Для заполнения пропущенных значений в столбце `total_income` выполним те же действия, что и в случае со столбцом `days_employed`.

In [21]:
def mean_total_income(row):
    years = row['dob_years']
    education = row['education']
    return df[(df['total_income'].isna() == False) \
                & (df['dob_years'] == years) \
                & (df['education'] == education)]['total_income'].mean()

# создадим новый столбец, как результат применения функции к каждой строке в датасете
df['mean_total_income'] = df.apply(mean_total_income, axis = 1)

# новый столбец, равный столбцу 'total_income', но с заполненными пропусками 
df['total_income_0'] = df['total_income'].fillna(0)

# записываем значения столбца 'total_income', как старые, в случае, если пропуска в столбце нет (.isna() возвращает 0)
# и как среднее значение дохода в месяц для клиентов такого же возраста с таким же образованием, в случае пропуска
df['total_income'] = \
df['total_income_0'] + df['total_income'].isna() * df['mean_total_income']

In [22]:
df.head(13)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,mean_days_employed,days_employed_0,mean_total_income,total_income_0
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,2374.265767,-8437.673028,243693.632170,253875.639453
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,-1263.661373,-4024.803754,161767.072002,112080.014102
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,339.142890,-5623.422610,168922.922098,145885.952297
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,1673.410241,-4124.747207,148047.386144,267628.550329
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,91415.560024,340266.072047,148317.316447,158616.077870
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,-1322.509097,-926.185831,204180.415473,255763.565419
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,7070.099300,-2879.202052,245714.930658,240525.971920
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,49954.425498,-152.779569,154577.479758,135823.934197
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,-2034.546455,-6929.865299,205725.048329,95856.832424
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,2425.889224,-2188.756445,160935.019231,144425.938277


---

In [23]:
# удалим ненужные столбцы
del df['mean_days_employed']
del df['days_employed_0']
del df['mean_total_income']
del df['total_income_0']

In [24]:
# заменим тип данных в столбце 'total_income' на целочисленный методом astype('int')
df['total_income'] = df['total_income'].astype('int')

# заменим тип данных в столбце 'days_employed' на целочисленный методом astype('int')
df['days_employed'] = df['days_employed'].astype('int')

In [25]:
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [26]:
# выведем общую информацию о таблице
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21518 entries, 0 to 21517
Data columns (total 12 columns):
children            21518 non-null int64
days_employed       21518 non-null int32
dob_years           21518 non-null int64
education           21518 non-null object
education_id        21518 non-null int64
family_status       21518 non-null object
family_status_id    21518 non-null int64
gender              21518 non-null object
income_type         21518 non-null object
debt                21518 non-null int64
total_income        21518 non-null int32
purpose             21518 non-null object
dtypes: int32(2), int64(5), object(5)
memory usage: 1.8+ MB


In [27]:
# выведем количество пропущенных значений в столбцах
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Все пропуски NaN заполнены. Числовые переменные записаны в виде целых чисел.

---

Проведем поиск дубликатов. Будем использовать метод duplicated(). 

In [28]:
# количество полных дубликатов
df.duplicated().sum()

71

Метод drop_duplicates() находит полные дубликаты - идентичные записи во всех столбцах. Однако, полностью одинаковые записи могут принадлежать разным людям, т.к. значения в некоторых столбцах - категория на выбор, а некоторые значения, которые могут быть действительно уникальными, ранее были заполнены средними значениями. Не будем удалять дубликаты.

---

Проведем анализ столбца с целями кредитов.

In [29]:
# список уникальных записей в столбце 'purpose' и их количество
df['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     776
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
операции с жильем                         653
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
недвижимость                              634
строительство собственной недвижимости    634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      623
строительство недвижимости                620
покупка своего жилья                      620
ремонт жилью                              611
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Запишем получившийся после предобработки датасет в отдельный файл.

In [30]:
df.to_csv(path + '02_credit_data_edited.csv', index=True)

Проведем лемматизацию целей с использованием подпроцесса. Зададим модуль `lemmas_count`, который затем будем запускать с помощью %%bash.

`lemmas_count.py`
```python
#!/usr/bin/python
# -*- coding: utf-8 -*-
import sys
import getopt
import pandas as pd
from pymystem3 import Mystem

if __name__ == "__main__":

	path = '/mnt/c/Users/pavel/Desktop/da/projects/datasets/'
    
    #считаем содержимое файла 02_credit_data_edited.csv (датасет после предобработки)
	df = pd.read_csv(path + '02_credit_data_edited.csv', index_col=0)
	
	m = Mystem()
	
	# проведем лемматизацию (приведение к словарной форме слова) 
	# всех записей столбца с целями кредитов
	purposes_lem = []
	for i in range(len(df)):
	    lemmas = m.lemmatize(df['purpose'][i])
	    purposes_lem = purposes_lem + lemmas

	# импортируем контейнер Counter
	from collections import Counter

	# подсчет одинаковых значений и сортировка по частоте использования
	lemmas_list = Counter(purposes_lem).most_common()

	# создадим датафрейм из одинаковых значений и их количества
	data = pd.DataFrame(lemmas_list, columns = ['Лемма', 'Количество'])
	
	# запись в файл
	data.to_csv(path + '02_lemmas_count.csv', index=True)

	print('Done!')
```

In [31]:
%%bash
python3 lemmas_count.py

Done!


In [32]:
# считаем из файла результат работы модуля lemmas_count
lemmas_count = pd.read_csv(path + '02_lemmas_count.csv', index_col=0)
lemmas_count

,Лемма,Количество
0,,33666
1,\n,21518
2,недвижимость,6364
3,покупка,5909
4,жилье,4471
5,автомобиль,4314
6,образование,4022
7,с,2924
8,операция,2610
9,свадьба,2347


Анализируя этот набор лемм и уникальные значения для столбца `purpose`, выделим несколько категорий для цели кредита:
- жилая недвижимость;
- коммерческая недвижимость;
- автомобиль;
- образование;
- свадьба.

Создадим новый столбец, в котором укажем одну из категорий целей кредита в зависимости от леммы слов в цели кредита. Зададим модуль `lemmas_category`, который затем будем запускать с помощью %%bash.

`lemmas_category.py`
```python
#!/usr/bin/python
# -*- coding: utf-8 -*-
import sys
import getopt
import pandas as pd
from pymystem3 import Mystem

if __name__ == "__main__":

	path = '/mnt/c/Users/pavel/Desktop/da/projects/datasets/'
	
	#считаем содержимое файла 02_credit_data_edited.csv (датасет после предобработки)
	df = pd.read_csv(path + '02_credit_data_edited.csv', index_col=0)
	
	m = Mystem()
	
	# создадим функцию, присваивающую ту или иную категорию в зависимости от содержания входной строки
	def category(string):
		lemmas = m.lemmatize(string) 
		for i in range(len(lemmas)): 
			if (lemmas[i] == 'жилье') | (lemmas[i] == 'недвижимость'):
				return 'жилая недвижимость'
			if (lemmas[i] == 'коммерческий'):
				return 'коммерческая недвижимость'
			if (lemmas[i] == 'автомобиль'):
				return 'автомобиль'
			if (lemmas[i] == 'образование'):
				return 'образование'
		return 'свадьба'

	# новый столбец, значения в котором - результат работы функции category,
	# принимающей на вход значения в столбце 'purpose'
	df['purpose_category'] = df['purpose'].apply(category)

	# запись в файл
	df.to_csv(path + '02_credit_data_lemmatized.csv', index=True)

    print('Done!')
```

In [33]:
%%bash
python3 lemmas_category.py

Done!


In [34]:
# считаем из файла результат работы модуля lemmas_category
data = pd.read_csv(path + '02_credit_data_lemmatized.csv', index_col=0)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилая недвижимость
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилая недвижимость
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба


Проверим результат работы функции, все ли цели верно распределены по категориям. Для этого создадим сводную таблицу.

In [35]:
# сводная таблица для анализа работы функции category(), например, выведем количество (функция count()) 
# записей в столбце 'debt', сгруппированных по стобцу 'gender', группировка данных при этом будет происходить 
# по столбцам 'purpose_category' и 'purpose'
data_pivot = data.pivot_table(index = ['purpose_category','purpose'], 
                              columns = 'gender', values = 'debt', aggfunc = 'count')
print(data_pivot)

gender                                                              F    M
purpose_category          purpose                                         
автомобиль                автомобили                              322  156
                          автомобиль                              338  156
                          на покупку автомобиля                   295  177
                          на покупку подержанного автомобиля      324  155
                          на покупку своего автомобиля            338  167
                          приобретение автомобиля                 320  142
                          свой автомобиль                         313  167
                          сделка с автомобилем                    299  156
                          сделка с подержанным автомобилем        319  170
жилая недвижимость        жилье                                   409  238
                          недвижимость                            435  199
                         

Видим, что все цели правильно распределены по выбранным категориям.

---

Разделим клиентов на 2 категории: с детьми и без. Добавим новый столбец `with_children` в котором запишем значения 0, если нет детей и 1 если есть дети с помощью функции `with_children()`, принимающей на вход кол-во детей из столбца `children`.

In [36]:
def with_children(number_of_children):
    if number_of_children == 0:
        return 0
    return 1

# создадим новый столбец как результат работы функции with_children(), 
# примененной к столбцу 'children'
data['with_children'] = data['children'].apply(with_children)

Проверим результат работы функции, для этого выведем сводную таблицу.

In [37]:
# сводная таблица: количество записей в столбце 'debt',
# сгруппированных по столбцу 'gender',
# группировка по столбцам 'children_type' и 'children'
data_pivot = data.pivot_table(index = ['with_children','children'], 
                              columns = 'gender', values = 'debt', aggfunc = 'count')
print(data_pivot)

gender                     F     M
with_children children            
0              0        9568  4576
1             -1          35    12
               1        3094  1723
               2        1258   796
               3         196   134
               4          28    13
               5           7     2
               20         47    29


Видим, что функция сработала правильно, клиентам без детей присвоен 0, клиентам с любым другим количеством детей присвоена 1.

Заметим, также, необычные значения в столбце `children`: -1 и 20, с достаточно большим числом записей. Предположим, что такая форма записи несет какой-то смысл, например -1 для клиентов, ожидающих ребенка, но ещё его не имеющего, а 20 для всех клиентов с количеством детей больше 5. Не станем удалять эти записи.

---

Рассмотрим доход в месяц. Узнаем минимальный и максимальный доход, затем попробуем выделить несколько категорий.

In [38]:
print(data['total_income'].min())
print(data['total_income'].max())

20667
2265604


Выделим 6 категорий дохода в месяц:
- `<100.000`;
- `100.000 - 200.000`;
- `200.000 - 350.000`;
- `350.000 - 500.000`;
- `500.000 - 1.000.000`;
- `>1.000.000`.

In [39]:
# создадим функцию, присваивающую ту или иную категорию в зависимости от содержания столбца 'total_income'
def category_income(string):
    if string < 100000:
        return '<100.000'
    if (string >= 100000) & (string < 200000):
        return '100.000 - 200.000'
    if (string >= 200000) & (string < 350000):
        return '200.000 - 350.000'
    if (string >= 350000) & (string < 500000):
        return '350.000 - 500.000'
    if (string >= 500000) & (string < 1000000):
        return '500.000 - 1.000.000'
    return '> 1.000.000'

In [40]:
# новый столбец, значения в котором - результат работы функции category_income, 
# получающей на вход значения столбца 'total_income'
data['income_category'] = data['total_income'].apply(category_income)

In [41]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,with_children,income_category
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилая недвижимость,1,200.000 - 350.000
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1,100.000 - 200.000
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилая недвижимость,0,100.000 - 200.000
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,1,200.000 - 350.000
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,0,100.000 - 200.000


In [42]:
# количество клиентов в категориях
data.groupby('income_category')['debt'].count()

income_category
100.000 - 200.000      11649
200.000 - 350.000       4552
350.000 - 500.000        635
500.000 - 1.000.000      197
<100.000                4460
> 1.000.000               25
Name: debt, dtype: int64

Количество записей в категориях существенно отличается, изменим границы категорий, чтобы добиться более равномерного распределения по категориям дохода.

In [43]:
def category_income_new(string):
    if string < 100000:
        return '<100.000'
    if (string >= 100000) & (string < 150000):
        return '100.000 - 150.000'
    if (string >= 150000) & (string < 200000):
        return '150.000 - 200.000'
    if (string >= 200000) & (string < 300000):
        return '200.000 - 300.000'
    return '> 300.000'

data['income_category'] = data['total_income'].apply(category_income_new)
print(data.groupby('income_category')['debt'].count())

income_category
100.000 - 150.000    6252
150.000 - 200.000    5397
200.000 - 300.000    3927
<100.000             4460
> 300.000            1482
Name: debt, dtype: int64


Имеем 4 группы дохода с сопоставимым количеством клиентов в каждой категории.

## 3. Влияние характеристик клиента на возврат долга
В этом разделе проанализируем зависимость возврата кредита в срок от разных характеристик. Будем использовать группировку по одному из признаков и вычислять среднее значение столбца `debt` для каждой выделенной группы.

В рамках этого исследования можно сделать предположение о том, что среднее значение для группы будет равно вероятности того, что новый клиент с таким же признаком не вернет кредит в срок (будет иметь задолженность). 

### 3.1 Наличие детей
Проанализируем зависимость возврата кредита в срок от наличия детей. Сгруппируем датасет по столбцу `with_children` и вычислим среднее значение столбца `debt` в полученных наборах. 

In [44]:
# сгруппируем по столбцу 'with_children', вычислим среднее значение столбца 'debt' для каждой группы, 
# отсортируем по убыванию
data.groupby('with_children')['debt'].mean().sort_values(ascending = False)

with_children
1    0.091673
0    0.075156
Name: debt, dtype: float64

Видим разные средние значения столбца `debt` для разных групп. Так, для клиентов с детьми вероятность иметь задолженность - 9,2 %. Для клиентов без детей - 7.5%.

Можно сделать вывод о том, что для банка безопаснее выдавать кредит клиентам, не имеющим детей.

### 3.2 Семейное положение
Проанализируем зависимость возврата кредита в срок от семейного положения. Сгруппируем датасет по столбцу `family_status` и вычислим среднее значение столбца `debt` в полученных наборах. 

In [45]:
# сгруппируем по столбцу 'family_status', вычислим среднее значение столбца 'debt' для каждой группы, 
# отсортируем по убыванию
data.groupby('family_status')['debt'].mean().sort_values(ascending = False)

family_status
Не женат / не замужем    0.097440
гражданский брак         0.092956
женат / замужем          0.075059
в разводе                0.071130
вдовец / вдова           0.065625
Name: debt, dtype: float64

Видим разные средние значения столбца `debt` для разных групп. Так, для клиентов не состоящих в браке вероятность иметь задолженность - 9,7 %. Для клиентов в браке - 7.5%.

Можно сделать вывод, о том, что для банка безопаснее выдавать кредит клиентам в браке/разводе/овдовевшим, чем клиентам, не состоящим в браке.

### 3.3 Суммарный доход
Проанализируем зависимость возврата кредита в срок от суммарного дохода в месяц. Сгруппируем датасет по столбцу `income_category` и вычислим среднее значение столбца `debt` в полученных наборах. 

In [46]:
# сгруппируем по столбцу 'income_category', вычислим среднее значение столбца 'debt' для каждой группы, 
# отсортируем по убыванию
data.groupby('income_category')['debt'].mean().sort_values(ascending = False)

income_category
150.000 - 200.000    0.088753
100.000 - 150.000    0.085413
<100.000             0.078924
> 300.000            0.071525
200.000 - 300.000    0.068245
Name: debt, dtype: float64

Видим разные средние значения столбца `debt` для разных групп. Так, для клиентов, имеющих доход 150.000-200.000, вероятность иметь задолженность - 8,8 %. Для клиентов, имеющих доход 200.000-300.000, - 6.8%.

Можно сделать вывод, о том, что для банка безопаснее выдавать кредит клиентам с уровнем дохода 200.000-300.000.

### 3.4 Цели кредита
Проанализируем зависимость возврата кредита в срок от цели кредита. Сгруппируем датасет по столбцу `purpose_category` и вычислим среднее значение столбца `debt` в полученных наборах.

In [47]:
# сгруппируем по столбцу 'purpose_category', вычислим среднее значение столбца 'debt' для каждой группы, 
# отсортируем по убыванию
data.groupby('purpose_category')['debt'].mean().sort_values(ascending = False)

purpose_category
автомобиль                   0.093185
образование                  0.091994
свадьба                      0.079250
коммерческая недвижимость    0.075285
жилая недвижимость           0.071639
Name: debt, dtype: float64

Видим разные средние значения столбца `debt` для разных групп. Так, для клиентов, оформляющих кредит на автомобиль, вероятность иметь задолженность - 9,3 %. Для клиентов, оформляющих кредит на жилую недвижимость, - 7.2%.

Можно сделать вывод, о том, что для банка безопаснее выдавать кредит на цели, связанные с недвижимостью.

## 4. Выводы

В ходе проведенного исследования были получены средние значения вероятности задолженности по кредиту для разных групп клиентов. Были выявлены наиболее безопасные с точки зрения кредитования категории клиентов: без детей, состоящие в браке/разводе, с доходом в месяц 200.000-300.000, оформляющих кредит на цели, связанные с недвижимостью.